# XRD Data Visualization: API & File Upload
This notebook allows you to visualize XRD data from two sources:
- **API**: Select batches and samples from the NOMAD HySprint API.
- **File Upload**: Upload local `.xy` files for visualization.

You can overlay, analyze, and detect peaks in the data interactively. Follow the cells below in order for a smooth workflow.

In [22]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import re
from typing import Dict, List, Tuple
import numpy as np
from scipy.signal import find_peaks, peak_widths
import sys
import os
import math

sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_ids_in_batch, get_sample_description, get_all_eqe as get_all_xrd
import batch_selection
import plotting_utils

# --- Authentication & API Setup ---
token = None
url_base = "https://nomad-hzb-se.de"
url = f"{url_base}/nomad-oasis/api/v1"

if 'NOMAD_CLIENT_ACCESS_TOKEN' in os.environ:
    token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']
else:
    try:
        import importlib.util
        import pathlib
        secrets_path = pathlib.Path(os.getcwd()).parent / 'secrets.py'
        spec = importlib.util.spec_from_file_location('secrets', str(secrets_path))
        secrets = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(secrets)
        token = getattr(secrets, 'TOKEN', None)
    except Exception:
        token = None
    if token is None:
        try:
            import access_token
            token = access_token.get_token(url)
        except Exception:
            token = None
if token is None:
    print("No token found in environment, secrets.py (one level above), or access_token. Please set one of these methods.")

In [23]:
# Authentication and API setup

token = None
url_base = "https://nomad-hzb-se.de"
url = f"{url_base}/nomad-oasis/api/v1"

if 'NOMAD_CLIENT_ACCESS_TOKEN' in os.environ:
    token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']
else:
    try:
        import importlib.util
        import pathlib
        secrets_path = pathlib.Path(os.getcwd()).parent / 'secrets.py'
        spec = importlib.util.spec_from_file_location('secrets', str(secrets_path))
        secrets = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(secrets)
        token = getattr(secrets, 'TOKEN', None)
    except Exception:
        token = None
    if token is None:
        try:
            import access_token
            token = access_token.get_token(url)
        except Exception:
            token = None
if token is None:
    print("No token found in environment, secrets.py (one level above), or access_token. Please set one of these methods.")

warning_sign = "\u26A0"

out = widgets.Output()
out2 = widgets.Output()
read = widgets.Output()
dynamic_content = widgets.Output()
results_content = widgets.Output(layout={
    'max_height': '1000px',
    'overflow': 'scroll',
    })
cell_edit = widgets.VBox() 

default_variables = widgets.Dropdown(
    options=['sample name', 'batch',"sample description", 'custom'],
    index=0,
    description='name preset:',
    disabled=False,
    tooltip="Presets for how the samples will be named in the plot"
)
data = None
original_data = None
display(dynamic_content)

Output()

In [24]:
def get_xrd_data(try_sample_ids, variation):
    try:
        all_xrd = get_all_xrd(url, token, try_sample_ids, eqe_type="HySprint_XRD_XY")
        existing_sample_ids = pd.Series(all_xrd.keys())
        if len(existing_sample_ids) == 0:
            return None
        sample_dict = {}
        for sample_id, sample_data in all_xrd.items():
            for xrd_entry in sample_data:
                df = pd.DataFrame(xrd_entry[0]["data"])
                angle = df.iloc[:, 0].to_numpy()
                intensity = df.iloc[:, 1].to_numpy()
                sample_dict[sample_id] = {
                    'angle': angle,
                    'intensity': intensity,
                    'variation': variation.get(sample_id, ''),
                    'name': xrd_entry[0].get("name", '')
                }
        return sample_dict if sample_dict else None
    except Exception as e:
        print(f"Error getting XRD data: {e}")
        return None

In [25]:
# Unified XRD Visualization Tool

class XRDVisualizationTool:
    def __init__(self, api_data=None):
        self.data_files = {}  # {filename: (x_data, y_data, metadata)}
        self.checkboxes = {}
        self.individual_plots = {}
        self.peak_controls = {}
        self.overlay_plot_output = widgets.Output()
        self.colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
        self.stagger_slider = widgets.FloatSlider(
            value=0.0, min=0.0, max=1000.0, step=10.0,
            description='Stagger Offset:', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px')
        )
        self.stagger_slider.observe(self.update_overlay_plot, names='value')
        self.api_data = api_data
        self.setup_ui()

    def parse_xy_file(self, file_content: str, filename: str):
        lines = file_content.strip().split('\n')
        metadata = {}
        
        # Check for metadata in first line
        if lines and lines[0].startswith("'Id:"):
            metadata_line = lines[0].strip("'")
            pattern = r'(\w+):\s*"([^"]*)"'
            matches = re.findall(pattern, metadata_line)
            metadata = dict(matches)
            lines = lines[1:]  # Remove metadata line
        
        x_data, y_data = [], []
        for line in lines:
            line = line.strip()
            if line and not line.startswith('#'):  # Skip empty lines and comments
                try:
                    parts = line.split()
                    if len(parts) >= 2:
                        x = float(parts[0])
                        y = float(parts[1])
                        x_data.append(x)
                        y_data.append(y)
                except ValueError:
                    continue
        
        return x_data, y_data, metadata

    def find_peaks_in_data(x_data, y_data, height_threshold=None, prominence=None):
        """Find peaks in XRD data"""
        y_array = np.array(y_data)
        x_array = np.array(x_data)
        
        if height_threshold is None:
            height_threshold = np.max(y_array) * 0.1
        if prominence is None:
            prominence = np.max(y_array) * 0.05
        
        peaks, properties = find_peaks(
            y_array, 
            height=height_threshold, 
            prominence=prominence, 
            distance=5
        )
        
        peak_positions = x_array[peaks].tolist()
        peak_intensities = y_array[peaks].tolist()
        
        return peaks, peak_positions, peak_intensities

    def create_peak_controls(self, filename):
        x_data, y_data, metadata = self.data_files[filename]
        max_intensity = float(np.max(y_data)) if len(y_data) > 0 else 1.0
        default_height = max_intensity * 0.1
        default_prominence = max_intensity * 0.05
        height_slider = widgets.FloatSlider(
            value=default_height,
            min=0.0,
            max=max_intensity,
            step=max(1.0, max_intensity/200),
            description='Min Height:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )
        prominence_slider = widgets.FloatSlider(
            value=default_prominence,
            min=0.0,
            max=max_intensity,
            step=max(1.0, max_intensity/200),
            description='Prominence:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='300px')
        )
        show_peaks_checkbox = widgets.Checkbox(
            value=True,
            description='Show Peaks',
            style={'description_width': 'initial'}
        )
        peak_info_output = widgets.Output(layout=widgets.Layout(height='100px'))
        def update_peaks(change=None):
            try:
                self.update_individual_plot_peaks(
                    filename, 
                    height_slider.value, 
                    prominence_slider.value, 
                    show_peaks_checkbox.value, 
                    peak_info_output
                )
            except Exception as e:
                with peak_info_output:
                    clear_output(wait=True)
                    print(f"Error updating peaks: {e}")

    def update_individual_plot_peaks(self, filename, height_threshold, prominence, show_peaks, peak_info_output):
        try:
            x_data, y_data, metadata = self.data_files[filename]
            plot_widget = self.individual_plots[filename]
            
            # Clear existing traces
            plot_widget.data = []
            
            # Add main data trace
            plot_widget.add_scatter(
                x=x_data, 
                y=y_data, 
                mode='lines', 
                name='Data', 
                line=dict(width=2, color='blue')
            )
            
            peak_info_text = "No peaks detected"
            
            if show_peaks and len(y_data) > 0:
                try:
                    peaks, peak_positions, peak_intensities = self.find_peaks_in_data(
                        x_data, y_data, height_threshold, prominence
                    )
                    
                    if len(peak_positions) > 0:
                        plot_widget.add_scatter(
                            x=peak_positions,
                            y=peak_intensities,
                            mode='markers',
                            name='Peaks',
                            marker=dict(color='red', size=8, symbol='triangle-up'),
                            hovertemplate='Peak at 2θ: %{x:.2f}°<br>Intensity: %{y:.1f}<extra></extra>'
                        )
                        
                        peak_info_lines = [f"Found {len(peak_positions)} peaks:"]
                        for i, (pos, intensity) in enumerate(zip(peak_positions, peak_intensities)):
                            peak_info_lines.append(f"Peak {i+1}: 2θ = {pos:.2f}°, I = {intensity:.1f}")
                        peak_info_text = "\n".join(peak_info_lines)
                except Exception as e:
                    peak_info_text = f"Error detecting peaks: {e}"
            
            with peak_info_output:
                clear_output(wait=True)
                print(peak_info_text)
                
        except Exception as e:
            with peak_info_output:
                clear_output(wait=True)
                print(f"Error updating plot: {e}")

    def set_stagger_slider_range(self):
        max_intensity = 1.0
        for x_data, y_data, metadata in self.data_files.values():
            if len(y_data) > 0:
                max_intensity = max(max_intensity, float(np.max(y_data)))
        
        if max_intensity > 0:
            slider_max = 10 ** math.ceil(math.log10(max_intensity))
        else:
            slider_max = 1000.0
        
        self.stagger_slider.max = slider_max
        self.stagger_slider.value = slider_max * 0.1
        self.stagger_slider.step = max(1.0, slider_max / 100)

    def load_api_data(self, api_data):
        print(f"Loading API data with {len(api_data)} samples")
        
        self.data_files = {}
        self.checkboxes = {}
        self.individual_plots = {}
        self.peak_controls = {}
        
        for sample_id, info in api_data.items():
            print(f"Processing sample: {sample_id}")
            print(f"  - Angle data points: {len(info['angle'])}")
            print(f"  - Intensity data points: {len(info['intensity'])}")
            
            x_data = info['angle']
            y_data = info['intensity']
            metadata = {
                'variation': info.get('variation', ''), 
                'name': info.get('name', ''), 
                'sample_id': sample_id
            }
            
            self.data_files[sample_id] = (x_data, y_data, metadata)
            
            # Create individual plot
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines', name=sample_id))
            fig.update_layout(
                title=f"Sample: {sample_id}<br>Variation: {metadata['variation']}<br>Name: {metadata['name']}",
                xaxis_title='2θ (degrees)',
                yaxis_title='Intensity',
                width=700,
                height=450,
                showlegend=True
            )
            
            plot_widget = go.FigureWidget(fig)
            self.individual_plots[sample_id] = plot_widget
            print(f"  - Created plot widget for {sample_id}")
            
            # Create checkbox
            checkbox = widgets.Checkbox(
                value=False,
                description=f'Include {sample_id}',
                style={'description_width': 'initial'}
            )
            checkbox.observe(self.update_overlay_plot, names='value')
            self.checkboxes[sample_id] = checkbox
            
            # Create peak controls
            try:
                peak_controls = self.create_peak_controls(sample_id)
                self.peak_controls[sample_id] = peak_controls
                print(f"  - Created peak controls for {sample_id}")
            except Exception as e:
                print(f"  - Error creating peak controls: {e}")
        
        self.set_stagger_slider_range()
        print("About to update display...")
        self.update_display()
        print("Display updated!")

    def on_file_upload(self, change):
        """Handle file upload"""
        uploaded_files = change['new']
        for file_obj in uploaded_files:
            filename = file_obj.name
            if filename.endswith('.xy'):
                try:
                    file_content = file_obj.content.decode('utf-8')
                    x_data, y_data, metadata = self.parse_xy_file(file_content, filename)
                    
                    if len(x_data) == 0 or len(y_data) == 0:
                        print(f"⚠️ No data found in file: {filename}")
                        continue
                    
                    self.data_files[filename] = (x_data, y_data, metadata)
                    
                    # Create individual plot
                    fig = go.Figure()
                    fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines', name=filename))
                    
                    title_parts = [f"File: {filename}"]
                    if 'Id' in metadata:
                        title_parts.append(f"ID: {metadata['Id']}")
                    if 'Operator' in metadata:
                        title_parts.append(f"Operator: {metadata['Operator']}")
                    
                    fig.update_layout(
                        title='<br>'.join(title_parts),
                        xaxis_title='2θ (degrees)',
                        yaxis_title='Intensity',
                        width=700,
                        height=450,
                        showlegend=True
                    )
                    
                    plot_widget = go.FigureWidget(fig)
                    self.individual_plots[filename] = plot_widget
                    
                    # Create checkbox
                    checkbox = widgets.Checkbox(
                        value=False,
                        description=f'Include {filename}',
                        style={'description_width': 'initial'}
                    )
                    checkbox.observe(self.update_overlay_plot, names='value')
                    self.checkboxes[filename] = checkbox
                    
                    # Create peak controls
                    peak_controls = self.create_peak_controls(filename)
                    self.peak_controls[filename] = peak_controls
                    
                except Exception as e:
                    print(f"Error processing file {filename}: {e}")
                    continue
        
        self.set_stagger_slider_range()
        self.update_display()

    def update_overlay_plot(self, change=None):
        with self.overlay_plot_output:
            clear_output(wait=True)
            selected_files = [filename for filename, checkbox in self.checkboxes.items() if checkbox.value]
            if not selected_files:
                print("No files selected for overlay plot")
                return
            fig = go.Figure()
            stagger_offset = self.stagger_slider.value
            for i, filename in enumerate(selected_files):
                x_data, y_data, metadata = self.data_files[filename]
                staggered_y_data = [y + (i * stagger_offset) for y in y_data]
                color = self.colors[i % len(self.colors)]
                display_name = filename
                if stagger_offset > 0:
                    display_name = f"{filename} (+{i * stagger_offset:.0f})"
                fig.add_trace(go.Scatter(
                    x=x_data,
                    y=staggered_y_data,
                    mode='lines',
                    name=display_name,
                    line=dict(color=color, width=2)
                ))
            title = 'Overlay Plot - Selected Files'
            if stagger_offset > 0:
                title += f' (Staggered by {stagger_offset})'
            fig.update_layout(
                title=title,
                xaxis_title='2θ (degrees)',
                yaxis_title='Intensity',
                width=900,
                height=600,
                showlegend=True,
                legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.01)
            )
            display(go.FigureWidget(fig))

    def update_display(self):
        # Don't clear output here, just update the content
        if not self.data_files:
            with self.main_output:
                clear_output(wait=True)
                print("No files uploaded yet. Please upload .xy files above.")
            return
        
        with self.main_output:
            clear_output(wait=True)
            print(f"Loaded {len(self.data_files)} files:")
            print("=" * 50)
            
            for filename in self.data_files.keys():
                print(f"\n{filename}:")
                display(self.checkboxes[filename])
                display(self.individual_plots[filename])
                if filename in self.peak_controls:
                    display(self.peak_controls[filename])
                print("-" * 50)

    def setup_ui(self):
        self.file_upload = widgets.FileUpload(
            accept='.xy', multiple=True, description='Upload .xy files'
        )
        self.file_upload.observe(self.on_file_upload, names='value')
        self.main_output = widgets.Output()
        instructions = widgets.HTML("""
        <h2>XRD Data Visualization Tool</h2>
        <p><strong>Instructions:</strong></p>
        <ol>
            <li>Option 1: Select batches from the API above. The samples will appear below for visualization.</li>
            <li>Option 2: If you have local .xy files, upload them below to visualize.</li>
            <li>Use checkboxes to select samples for overlay, and adjust the stagger offset as needed.</li>
        </ol>
        """)
        display(instructions)
        display(self.file_upload)
        display(self.main_output)
        stagger_section = widgets.HTML("""
        <h3>Overlay Plot Controls:</h3>
        <p>Adjust the stagger offset to vertically separate curves in the overlay plot:</p>
        """)
        print("\nOverlay Plot:")
        print("=" * 50)
        display(stagger_section)
        display(self.stagger_slider)
        display(self.overlay_plot_output)
        self.update_display()

def launch_xrd_visualization(api_data=None):
    if api_data:
        # Create main container
        main_container = widgets.VBox()
        
        # Create checkboxes for overlay selection
        checkboxes = {}
        plot_containers = {}
        
        # Create individual plot containers
        for sample_id, info in api_data.items():
            # Create checkbox
            checkbox = widgets.Checkbox(
                value=False,
                description=f'Include {sample_id} in overlay',
                style={'description_width': 'initial'}
            )
            checkboxes[sample_id] = checkbox
            
            # Create plot
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=info['angle'], 
                y=info['intensity'], 
                mode='lines', 
                name=sample_id
            ))
            
            # Add peaks to plot
            try:
                y_data = info['intensity']
                x_data = info['angle']
                max_intensity = float(np.max(y_data))
                height_threshold = max_intensity * 0.1
                prominence = max_intensity * 0.05
                
                peaks, properties = find_peaks(
                    y_data, 
                    height=height_threshold, 
                    prominence=prominence, 
                    distance=5
                )
                
                if len(peaks) > 0:
                    peak_positions_deg = x_data[peaks]
                    peak_intensities_val = y_data[peaks]
                    
                    # Add peaks to plot
                    fig.add_trace(go.Scatter(
                        x=peak_positions_deg,
                        y=peak_intensities_val,
                        mode='markers',
                        name='Peaks',
                        marker=dict(color='red', size=8, symbol='triangle-up'),
                        hovertemplate='Peak at 2θ: %{x:.2f}°<br>Intensity: %{y:.1f}<extra></extra>'
                    ))
                    
                    peak_info = f"Found {len(peaks)} peaks: " + ", ".join([f"{pos:.2f}°" for pos in peak_positions_deg])
                else:
                    peak_info = "No peaks detected"
            except Exception as e:
                peak_info = f"Peak detection error: {e}"
            
            fig.update_layout(
                title=f"Sample: {sample_id}<br>Variation: {info.get('variation', '')}<br>Name: {info.get('name', '')}<br>{peak_info}",
                xaxis_title='2θ (degrees)',
                yaxis_title='Intensity',
                width=800,
                height=500,
                showlegend=True
            )
            
            plot_widget = go.FigureWidget(fig)
            
            # Create container for this sample
            sample_container = widgets.VBox([
                widgets.HTML(f"<h3>{sample_id}</h3>"),
                checkbox,
                plot_widget,
                widgets.HTML("<hr>")
            ])
            
            plot_containers[sample_id] = sample_container
        
        # Create overlay controls
        stagger_slider = widgets.FloatSlider(
            value=0.0, min=0.0, max=10000.0, step=100.0,
            description='Stagger Offset:', 
            style={'description_width': 'initial'}, 
            layout=widgets.Layout(width='400px')
        )
        
        overlay_button = widgets.Button(
            description='Create Overlay Plot',
            button_style='success'
        )
        
        overlay_output = widgets.Output()
        
        def create_overlay(b):
            with overlay_output:
                overlay_output.clear_output(wait=True)
                
                selected_samples = [sample_id for sample_id, checkbox in checkboxes.items() if checkbox.value]
                
                if not selected_samples:
                    print("Please select at least one sample using the checkboxes above")
                    return
                
                fig = go.Figure()
                colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
                stagger_offset = stagger_slider.value
                
                for i, sample_id in enumerate(selected_samples):
                    info = api_data[sample_id]
                    staggered_y = info['intensity'] + (i * stagger_offset)
                    color = colors[i % len(colors)]
                    
                    display_name = sample_id
                    if stagger_offset > 0:
                        display_name = f"{sample_id} (+{i * stagger_offset:.0f})"
                    
                    fig.add_trace(go.Scatter(
                        x=info['angle'],
                        y=staggered_y,
                        mode='lines',
                        name=display_name,
                        line=dict(color=color, width=2)
                    ))
                
                title = f'Overlay Plot - {len(selected_samples)} samples'
                if stagger_offset > 0:
                    title += f' (Staggered by {stagger_offset})'
                
                fig.update_layout(
                    title=title,
                    xaxis_title='2θ (degrees)',
                    yaxis_title='Intensity',
                    width=1000,
                    height=700,
                    showlegend=True
                )
                
                display(go.FigureWidget(fig))
        
        overlay_button.on_click(create_overlay)
        
        # Assemble the complete interface
        header = widgets.HTML(f"<h2>XRD Data Visualization - {len(api_data)} samples loaded</h2>")
        
        # Individual plots section
        individual_plots = widgets.VBox(
            [widgets.HTML("<h3>Individual Plots:</h3>")] + 
            list(plot_containers.values())
        )
        
        # Overlay section
        overlay_section = widgets.VBox([
            widgets.HTML("<h3>Create Overlay Plot:</h3>"),
            widgets.HTML("<p>Select samples using the checkboxes above, then click the button below:</p>"),
            stagger_slider,
            overlay_button,
            overlay_output
        ])
        
        # Complete interface
        complete_interface = widgets.VBox([
            header,
            individual_plots,
            overlay_section
        ])
        
        return complete_interface
        
    else:
        # File upload mode for Voila
        file_upload = widgets.FileUpload(
            accept='.xy', 
            multiple=True, 
            description='Upload .xy files'
        )
        
        upload_output = widgets.Output()
        
        def handle_upload(change):
            with upload_output:
                upload_output.clear_output(wait=True)
                
                uploaded_files = change['new']
                if not uploaded_files:
                    return
                
                file_plots = []
                
                for file_obj in uploaded_files:
                    filename = file_obj.name
                    if filename.endswith('.xy'):
                        try:
                            file_content = file_obj.content.decode('utf-8')
                            lines = file_content.strip().split('\n')
                            
                            x_data, y_data = [], []
                            for line in lines:
                                line = line.strip()
                                if line and not line.startswith('#') and not line.startswith("'"):
                                    try:
                                        parts = line.split()
                                        if len(parts) >= 2:
                                            x = float(parts[0])
                                            y = float(parts[1])
                                            x_data.append(x)
                                            y_data.append(y)
                                    except ValueError:
                                        continue
                            
                            if len(x_data) > 0 and len(y_data) > 0:
                                fig = go.Figure()
                                fig.add_trace(go.Scatter(
                                    x=x_data, 
                                    y=y_data, 
                                    mode='lines', 
                                    name=filename
                                ))
                                fig.update_layout(
                                    title=f"File: {filename}",
                                    xaxis_title='2θ (degrees)',
                                    yaxis_title='Intensity',
                                    width=800,
                                    height=500
                                )
                                
                                file_plots.append(widgets.VBox([
                                    widgets.HTML(f"<h3>{filename}</h3>"),
                                    go.FigureWidget(fig)
                                ]))
                        
                        except Exception as e:
                            file_plots.append(widgets.HTML(f"<p>Error processing {filename}: {e}</p>"))
                
                if file_plots:
                    display(widgets.VBox(file_plots))
                else:
                    print("No valid .xy files found")
        
        file_upload.observe(handle_upload, names='value')
        
        file_interface = widgets.VBox([
            widgets.HTML("<h2>XRD File Upload</h2>"),
            widgets.HTML("<p>Upload your XRD .xy files below. Each file will be plotted individually.</p>"),
            file_upload,
            upload_output
        ])
        
        display(file_interface)
    
    return None

# --- Callback: Load Data from Batch Selection ---
def on_load_data_clicked(batch_ids_selector):
    global data, original_data
    with out:
        out.clear_output()
        print("Loading Data")
        try:
            try_sample_ids = get_ids_in_batch(url, token, batch_ids_selector.value)
            identifiers = get_sample_description(url, token, list(try_sample_ids))
            data = get_xrd_data(try_sample_ids, identifiers)
            if data is None:
                out.clear_output()
                print("The batches selected don't contain any XRD measurements")
                return
            original_data = data.copy()
            out.clear_output()
            print("Data Loaded")
            
            # Create and display the widget directly
            main_widget = launch_xrd_visualization(data)
            if main_widget:
                display(main_widget)
            
        except Exception as e:
            out.clear_output()
            print(f"Error loading data: {e}")

In [26]:
# --- Batch Selection Widget with Optional Filtering ---
def create_batch_selection_with_optional_filtering():
    original_batch_widget = batch_selection.create_batch_selection(url, token, on_load_data_clicked)
    batch_selector = None
    for child in original_batch_widget.children:
        if isinstance(child, widgets.SelectMultiple):
            batch_selector = child
            break
    total_batches = len(batch_selector.options) if batch_selector else 0
    filter_button = widgets.Button(
        description=f"🔍 Filter to show only batches with XRD data",
        button_style='info',
        tooltip=f'Click to filter {total_batches} batches (this may take a few minutes)',
        layout=widgets.Layout(width='400px')
    )
    filter_status = widgets.Output()
    def start_filtering(b):
        filter_button.disabled = True
        filter_button.description = "🔄 Filtering in progress..."
        with filter_status:
            filter_status.clear_output(wait=True)
            print("Finding batches with XRD data...")
            batch_ids_list_tmp = list(get_batch_ids(url, token))
            all_batch_ids = []
            for batch in batch_ids_list_tmp:
                if "_".join(batch.split("_")[:-1]) in batch_ids_list_tmp:
                    continue
                all_batch_ids.append(batch)
            print(f"Testing {len(all_batch_ids)} batches...")
            valid_batches = []
            for i, batch_id in enumerate(all_batch_ids):
                if i % 10 == 0 or i == len(all_batch_ids) - 1:
                    filter_status.clear_output(wait=True)
                    print(f"Progress: {i+1}/{len(all_batch_ids)} - Found {len(valid_batches)} valid batches")
                    print(f"Currently testing: {batch_id}")
                try:
                    sample_ids = get_ids_in_batch(url, token, [batch_id])
                    if sample_ids:
                        identifiers = get_sample_description(url, token, list(sample_ids))
                        xrd_data = get_xrd_data(sample_ids, identifiers)
                        if xrd_data is not None:
                            valid_batches.append(batch_id)
                            filter_status.clear_output(wait=True)
                            print(f"✅ Found valid batch: {batch_id} ({len(xrd_data)} rows)")
                            print(f"Total found so far: {len(valid_batches)}")
                except:
                    continue
            if batch_selector:
                batch_selector.options = valid_batches
            filter_status.clear_output(wait=True)
            print("="*60)
            print("FILTERING COMPLETE")
            print("="*60)
            print(f"✅ Found {len(valid_batches)} batches with XRD data out of {total_batches} total")
            if len(valid_batches) > 0:
                print(f"Valid batches: {valid_batches}")
            else:
                print("⚠️  No batches with XRD data found!")
            filter_button.description = f"✅ Filtering complete - {len(valid_batches)} valid batches found"
            filter_button.disabled = True
            info_html = widgets.HTML(
                value=f"<p><b>Showing {len(valid_batches)} of {total_batches} batches with confirmed AbsPL data</b></p>"
            )
            original_batch_widget.children = (info_html,) + original_batch_widget.children
    filter_button.on_click(start_filtering)
    complete_widget = widgets.VBox([
        widgets.HTML(f"<p>Select batches from all {total_batches} available batches, or use the filter button below:</p>"),
        filter_button,
        filter_status,
        original_batch_widget
    ])
    return complete_widget

In [27]:
# --- MAIN EXECUTION ---
try:
    display(plotting_utils.create_manual("eqe_manual.md"))
except:
    print("Manual not available")

try:
    batch_widget = create_batch_selection_with_optional_filtering()
    display(batch_widget)
except Exception as e:
    print(f"Could not create batch selection: {e}")
    print("Proceeding with file upload mode only.")

# For file upload mode when no API data is loaded
file_widget = launch_xrd_visualization()
if file_widget:
    with dynamic_content:
        display(file_widget)